In [6]:
import requests
import pandas as pd
import json
import time

In [8]:
api_key = "YkbthBL7RWYK7M5OL-oLBstZGuZ2ByzGi0BxvA_VIGkC3Ui7JbS44CsCD_Olm1p6PFwUzuo6RfDFPdZ3phgA-xWqPDx4sCHZTeHgAlLxZLX60vgjMMlqS5IBJDCiZHYx"
endpoint = "businesses/search"
url = f"https://api.yelp.com/v3/{endpoint}"
headers = {
    "Authorization": f"Bearer {api_key}"
}
params = {
    "term": "restaurants",
    "location": "Louisville, KY",
    "limit": 50  # Set the desired limit per request (maximum is 50)
}

restaurant_data = []  # List to store the extracted data

offset = 0  # Initial offset value
results_per_request = params["limit"]  # Results per request (50)
total_results = float("inf")  # Initialize total_results to an arbitrary high value

while offset < total_results:
    params["offset"] = offset  # Set the offset parameter
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        break
    
    data = response.json()

    if offset == 0:
        total_results = data["total"]
    
    businesses = data["businesses"]
    
    for business in businesses:
        name = business.get("name")
        rating = business.get("rating")
        review_count = business.get("review_count")
        price = business.get("price")
        address = ", ".join(business.get("location", {}).get("display_address", []))
        restaurant_data.append({
            "Name": name,
            "Rating": rating,
            "Review Count": review_count,
            "Price": price,
            "Address": address
        })
    offset += results_per_request  
    time.sleep(2)  

# Create a DataFrame from the extracted data
df = pd.DataFrame(restaurant_data)

# Print the DataFrame
df.head()

Error: 400 - {"error": {"code": "VALIDATION_ERROR", "description": "Too many results requested, limit+offset must be <= 1000."}}


,Name,Rating,Review Count,Price,Address
0,The Eagle,4.5,1551,$$,"1314 Bardstown Rd, Louisville, KY 40204"
1,Hammerheads,4.5,1336,$$,"921 Swan St, Louisville, KY 40204"
2,Copper & Kings Rooftop Bar & Restaurant,4.5,124,$$,"1121 E Washington St, Copper and Kings, Louisv..."
3,Oliver's Chop House & Bourbon Bar,5.0,8,None,"4520 Poplar Level Rd, Derby City Gaming Hotel,..."
4,Pig Beach BBQ,3.5,14,None,"1201 River Rd, Louisville, KY 40206"


In [3]:
# df.to_csv('yelp.csv')

In [9]:
df = df.sort_values(by="Rating", ascending=True)
df = df.loc[df["Rating"] != 0.0]
df = df.reset_index(drop=True)
df.to_json('../json_files/yelp.json')